<a href="https://colab.research.google.com/github/darcor99/Python-Fraud-Analysis/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [4]:
!kaggle datasets download -d kartik2112/fraud-detection

Dataset URL: https://www.kaggle.com/datasets/kartik2112/fraud-detection
License(s): CC0-1.0
fraud-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!ls -l

!unzip fraud-detection.zip

total 206808
-rw-r--r-- 1 root root 211766662 Aug  5  2020 fraud-detection.zip
drwxr-xr-x 1 root root      4096 Oct 24 13:20 sample_data
Archive:  fraud-detection.zip
  inflating: fraudTest.csv           
  inflating: fraudTrain.csv          


In [5]:
!ls -l

total 4
drwxr-xr-x 1 root root 4096 Oct 24 13:20 sample_data


In [6]:
TrainingSet = '/content/fraudTrain.csv'
TestSet = '/content/fraudTest.csv'

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
!pip3 install pandasql
import pandasql as ps
from pandasql import sqldf
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=6f4f4918805860f6c23d2f8a9347652c47664ea84629cf3d598a45b1e04d4282
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [8]:
Train_df=pd.read_csv(TrainingSet)
Test_df = pd.read_csv(TestSet)

In [9]:
print(Train_df.head(10));

Train_df.columns

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   
5           5   2019-01-01 00:04:08  4767265376804500   
6           6   2019-01-01 00:04:42    30074693890476   
7           7   2019-01-01 00:05:08  6011360759745864   
8           8   2019-01-01 00:05:18  4922710831011201   
9           9   2019-01-01 00:06:01  2720830304681674   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy 

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [14]:
print('training set rows:= {}'.format(len(Train_df.index)));
print('test set rows:= {}'.format(len(Test_df.index)));


training set rows:= 1296675
test set rows:= 555719


In [23]:
Train_df.info(), print('\n')
Train_df.is_fraud.unique(), print('\n')
Train_df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

,0
Unnamed: 0,int64
trans_date_trans_time,object
cc_num,int64
merchant,object
category,object
amt,float64
first,object
last,object
gender,object
street,object


In [36]:
##need to drop unnecessary variables before doing log regression

##dropped many of the non-int/non-float columns for the training and test sets


##make a column in test and training sets for difference between merchant lat,long and user lat,long

X_train = Train_df.drop(columns = ['is_fraud', 'Unnamed: 0', 'cc_num', 'trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'])
y_train = Train_df['is_fraud']

X_test = Test_df.drop(columns = ['is_fraud', 'Unnamed: 0',  'cc_num', 'trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'])
y_test = Test_df['is_fraud']

#make columns for difference in lat/long, and --try make a column for number of transactions done in 1 day (if possible,... hard to do with unix time, might have to build timeseries model for this)
X_train['lat_dif'] = abs(X_train['merch_lat'] - X_train['lat'])
X_train['long_dif'] = abs(X_train['merch_long'] - X_train['long'])

X_test['lat_dif'] = abs(X_test['merch_lat'] - X_test['lat'])
X_test['long_dif'] = abs(X_test['merch_long'] - X_test['long'])



In [37]:
LogRgr = LogisticRegression()
LogRgr.fit(X_train, y_train)

LogisticRegression()

In [38]:
Y_Pred = LogRgr.predict(X_test)

In [28]:
LogRgr.score(X_test, y_test) ##99.6% accuracy -- not bad!

0.9961401355721147

In [ ]:
print((X_train['lat_dif']))
print(X_train['long_dif'])


print((X_train['lat']))
print(X_train['merch_lat'])

In [40]:
coef = LogRgr.coef_[0]
print(X_train.columns)
print(coef)



Index(['amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat',
       'merch_long', 'lat_dif', 'long_dif'],
      dtype='object')
[ 2.75030917e-03 -1.22091042e-06  8.22949724e-07 -3.72839145e-07
  6.49137215e-08 -4.05241991e-09  7.66672287e-07 -3.70573434e-07
  1.53215095e-08 -4.22724046e-09]


In [22]:
LogRgr.predict_proba(X_test)

array([[0.50073993, 0.49926007],
       [0.5011539 , 0.4988461 ],
       [0.50116203, 0.49883797],
       ...,
       [0.50194146, 0.49805854],
       [0.50000132, 0.49999868],
       [0.50134691, 0.49865309]])

In [41]:
X_train2 = Train_df.drop(columns = ['is_fraud', 'Unnamed: 0', 'cc_num', 'trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'])
y_train2 = Train_df['is_fraud']

X_test2 = Test_df.drop(columns = ['is_fraud', 'Unnamed: 0',  'cc_num', 'trans_date_trans_time', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'])
y_test2 = Test_df['is_fraud']

#make columns for difference in lat/long, and --try make a column for number of transactions done in 1 day (if possible,... hard to do with unix time, might have to build timeseries model for this)
X_train2['lat_dif'] = abs(X_train['merch_lat'] - X_train['lat'])
X_train2['long_dif'] = abs(X_train['merch_long'] - X_train['long'])

X_test2['lat_dif'] = abs(X_test['merch_lat'] - X_test['lat'])
X_test2['long_dif'] = abs(X_test['merch_long'] - X_test['long'])


X_train2 = X_train2.drop(columns = ['zip', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop'])
X_test2 = X_test2.drop(columns = ['zip', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop'])

In [42]:
LogReg2 = LogisticRegression()
LogReg2.fit(X_train2, y_train2)

LogisticRegression()

In [43]:
LogReg2.score(X_test2, y_test2)

0.9961401355721147